In [1]:
import os
import SimpleITK as sitk
import glob
import monai
from monai.transforms import (

    AsDiscrete,
    RandAdjustContrastd,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    AddChanneld,
    SpatialPadd,
    RandRotate90d,
    RandShiftIntensityd,
    EnsureTyped,
    EnsureType,
    MapTransform,
    Resized,
    Invertd,
    ToTensord,
    NormalizeIntensityd,
    RandFlipd,
    Lambdad,
    Activations,
    AsDiscrete,
)
from monai.metrics import ROCAUCMetric
from monai.data import CacheDataset, ThreadDataLoader,DataLoader, Dataset, decollate_batch,load_decathlon_datalist
import torch
import torch.nn as  nn
from torch.nn import Linear,  Softmax
import torch.nn.functional as F
from monai.utils import first, set_determinism
from random import shuffle, seed
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('./log/tensorboard')

In [2]:
torch.multiprocessing.set_sharing_strategy('file_system')
set_determinism(seed=1)

In [3]:
syf1_adcdir = '/app/liucd/deeplearn_dec/DL_dec/data_adc/syf_stage1/Mixed'
syf2_adcdir = '/app/liucd/deeplearn_dec/DL_dec/data_adc/syf_stage2/Mixed'
zy_adcdir = '/app/liucd/deeplearn_dec/DL_dec/data_adc/zunyi/Mixed'

syf1_dcedir = '/app/liucd/deeplearn_dec/DL_dec/data/syf_stage1/Mixed'
syf2_dcedir = '/app/liucd/deeplearn_dec/DL_dec/data/syf_stage2/Mixed'
zy_dcedir = '/app/liucd/deeplearn_dec/DL_dec/data/zunyi/Mixed'

adc_images = sorted(glob.glob(os.path.join(syf1_adcdir,  '*.nii.gz'))) + \
             sorted(glob.glob(os.path.join(syf2_adcdir,  '*.nii.gz'))) + \
             sorted(glob.glob(os.path.join(zy_adcdir,  '*.nii.gz'))) 

dce_images = sorted(glob.glob(os.path.join(syf1_dcedir,  '*.nii.gz'))) + \
             sorted(glob.glob(os.path.join(syf2_dcedir,  '*.nii.gz'))) + \
             sorted(glob.glob(os.path.join(zy_dcedir,  '*.nii.gz')))

seed(1)
shuffle(adc_images)
seed(1)
shuffle(dce_images)

splits =  int(0.333 * len(adc_images))

train_adcimages = adc_images[splits:]
val_adcimages   = adc_images[:splits]

train_dceimages = dce_images[splits:]
val_dceimages   = dce_images[:splits]


train_dict = [{'image_adc': image_adc, 'image_dce': image_dce,  'label': int(image_adc.split('_')[-1].replace('.nii.gz', ''))} 
                  for image_adc, image_dce in zip(train_adcimages, train_dceimages)]
val_dict = [{'image_adc': image_adc, 'image_dce': image_dce,  'label': int(image_adc.split('_')[-1].replace('.nii.gz', ''))} 
                  for image_adc, image_dce in zip(val_adcimages, val_dceimages)]

print(train_dict[-1])
len(train_dict), len(val_dict), len(train_dict + val_dict)


{'image_adc': '/app/liucd/deeplearn_dec/DL_dec/data_adc/syf_stage1/Mixed/2019_02_18_5096280_ADC2_0000_0.nii.gz', 'image_dce': '/app/liucd/deeplearn_dec/DL_dec/data/syf_stage1/Mixed/2019_02_18_5096280_+C2_0000_0.nii.gz', 'label': 0}


(438, 218, 656)

In [4]:
train_transforms = Compose(
        [
            LoadImaged(keys=["image_adc", "image_dce"]),
            EnsureChannelFirstd(keys=["image_adc", "image_dce"]),
            Orientationd(keys=["image_adc", 'image_dce'], axcodes="RAS"),
            Resized(keys=["image_adc"], spatial_size=(64, 64, 16)),
            Resized(keys=["image_dce"], spatial_size=(96, 96, 32)),
            
            NormalizeIntensityd(keys=["image_adc", "image_dce"], nonzero=True, channel_wise=True),
            
            RandFlipd( keys=["image_adc", ], spatial_axis=[0], prob=0.50),
            RandFlipd( keys=["image_adc", ], spatial_axis=[1], prob=0.50),
            RandFlipd( keys=["image_adc", ], spatial_axis=[2], prob=0.50),
            
            RandFlipd( keys=["image_dce", ], spatial_axis=[0], prob=0.50),
            RandFlipd( keys=["image_dce", ], spatial_axis=[1], prob=0.50),
            RandFlipd( keys=["image_dce", ], spatial_axis=[2], prob=0.50),
            
            RandRotate90d(keys=["image_adc", 'image_dce'], prob=0.50, max_k=3 ),
            RandShiftIntensityd( keys=["image_adc", 'image_dce'], offsets=0.10, prob=0.50),
            
            ToTensord(keys=['image_adc', 'image_dce', 'label'])
        ]
    )

val_transforms = Compose(
        [
            LoadImaged(keys=["image_adc",'image_dce' ]),
            EnsureChannelFirstd(keys=["image_adc", 'image_dce']),
            Orientationd(keys=["image_adc",'image_dce'], axcodes="RAS"),
            Resized(keys=["image_adc"], spatial_size=(64, 64, 16)),
            Resized(keys=["image_dce"], spatial_size=(96, 96, 32)),
            
            NormalizeIntensityd(keys=["image_adc", 'image_dce'], nonzero=True, channel_wise=True),
            ToTensord(keys=['image_adc', 'image_dce', 'label'])
        ]
    )


train_ds = CacheDataset(data=train_dict, transform=train_transforms, cache_rate=1.0, num_workers=12)
val_ds = CacheDataset(data=val_dict, transform=val_transforms, cache_rate=1.0, num_workers=12)

Loading dataset: 100%|███████████████████████████████████████████████████████| 218/218 [00:16<00:00, 13.15it/s]


In [5]:
# create a training data loader
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=16, pin_memory=True)

# create a validation data loader
val_loader = DataLoader(val_ds, batch_size=8, num_workers=8, pin_memory=True)

In [6]:
# test_net = monai.networks.nets.resnet34(spatial_dims=3, n_input_channels=1, num_classes=2, feed_forward=False)

for i in train_loader:
    print(i['image_adc'].shape, i['image_dce'].shape)   # 测试能否正常load数据
    print(i['image_adc_meta_dict']['filename_or_obj'])
    print(i['image_dce_meta_dict']['filename_or_obj'])
    print(i['label'])
    inputs = i['image_adc']
#     outputs = test_net(inputs)
#     print(outputs.shape)
    break

torch.Size([8, 1, 64, 64, 16]) torch.Size([8, 1, 96, 96, 32])
['/app/liucd/deeplearn_dec/DL_dec/data_adc/zunyi/Mixed/2020_10_09_1647391_ADC2_0000_1.nii.gz', '/app/liucd/deeplearn_dec/DL_dec/data_adc/zunyi/Mixed/2022_03_10_1744779_ADC2_0000_1.nii.gz', '/app/liucd/deeplearn_dec/DL_dec/data_adc/zunyi/Mixed/2021_08_09_1732329_ADC2_0000_1.nii.gz', '/app/liucd/deeplearn_dec/DL_dec/data_adc/zunyi/Mixed/2017_12_28_1378614_ADC2_0000_0.nii.gz', '/app/liucd/deeplearn_dec/DL_dec/data_adc/zunyi/Mixed/2020_04_27_1602872_ADC2_0000_1.nii.gz', '/app/liucd/deeplearn_dec/DL_dec/data_adc/zunyi/Mixed/2019_07_15_1534792_ADC2_0000_0.nii.gz', '/app/liucd/deeplearn_dec/DL_dec/data_adc/syf_stage2/Mixed/2020_08_12_15_57_5832173_ADC2_0000_1.nii.gz', '/app/liucd/deeplearn_dec/DL_dec/data_adc/zunyi/Mixed/2014_04_04_1006675_ADC2_0000_0.nii.gz']
['/app/liucd/deeplearn_dec/DL_dec/data/zunyi/Mixed/2020_10_09_1647391_+C2_0000_1.nii.gz', '/app/liucd/deeplearn_dec/DL_dec/data/zunyi/Mixed/2022_03_10_1744779_+C2_0000_1.nii.

In [8]:

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class DoubleTower(nn.Module):
    def __init__(self, 
                 pretrained_dce='', 
                 pretrained_adc='', 
                 device = torch.device("cuda"),
                 num_classes=2, 
                 fc_hidden_size = 128
                ):
        super().__init__()
        self.pretrained_dce = pretrained_dce
        self.pretrained_adc = pretrained_adc
        self.fc_hidden_size = fc_hidden_size
        self.num_classes = num_classes
        self.device = device
        
        self.model_dce = monai.networks.nets.resnet34(spatial_dims=3, n_input_channels=1, num_classes=2, feed_forward=False).to(self.device)
        self.model_adc = monai.networks.nets.resnet34(spatial_dims=3, n_input_channels=1, num_classes=2, feed_forward=False).to(self.device)
        
        if  pretrained_dce != '':
            dce_dict = self.model_dce.state_dict()
            dce_pretrain = torch.load(self.pretrained_dce, map_location=self.device)
            dce_pretrain_dict = {k:v for k, v in dce_pretrain.items() if  k in  dce_dict.keys()}
            dce_dict.update(dce_pretrain_dict)
            self.model_dce.load_state_dict(dce_dict)

        if  pretrained_adc !='':
            adc_dict = self.model_adc.state_dict()
            adc_pretrain = torch.load(self.pretrained_adc, map_location=self.device)
            adc_pretrain_dict = {k:v for k, v in adc_pretrain.items() if  k in  adc_dict.keys()}
            adc_dict.update(adc_pretrain_dict)
            self.model_adc.load_state_dict(adc_dict)
        
        
        self.Linear1 = Linear(1024, self.num_classes, device=self.device) 
        # self.Linear1 = Linear(512, self.fc_hidden_size, device=self.device)  # 1024 是 所有下采样特征图globalpool之后拼接的结果        
        # self.Linear2 = Linear(self.fc_hidden_size, self.num_classes, device=self.device)  
        
    
    def forward(self, x1, x2):  # x 是SegResNet的输入影像矩阵
        
        encode_output1 = self.model_dce(x1)
        encode_output2 = self.model_dce(x2)
        
        concatenated = torch.concat([encode_output1, encode_output2], dim=-1)
        # concatenated =  encode_output1 * encode_output2
        # fc1 = F.relu(self.Linear1(concatenated))
        # fc2 = self.Linear2(fc1)
        fc2 = self.Linear1(concatenated)
        return F.log_softmax(fc2, dim=-1)

# dce_pretrain_path = '../dce_225/best_metric_model_classification3d_dict.pth'
# adc_pretrain_path = '../adc_225/best_metric_model_classification3d_dict.pth'

# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

my_model = DoubleTower()

x1 = torch.randn(8, 1, 96, 96, 32)  # batch, channel, x, y, z
x2 = torch.randn(8, 1, 94, 64, 16)

output = my_model(x1.cuda(), x2.cuda())
print('output: ', output.shape)


output:  torch.Size([8, 2])


In [9]:
dce_pretrain_path = ''
adc_pretrain_path = ''

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = DoubleTower(dce_pretrain_path, adc_pretrain_path, device = device)
# model = DoubleTower(device = device)

In [12]:
post_pred = Compose([Activations(softmax=True)])
post_label = Compose([AsDiscrete(to_onehot=2)])

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
auc_metric = ROCAUCMetric()

# start a typical PyTorch training
val_interval = 1
best_metric = -1
best_metric_epoch = -1
max_epochs = 150
for epoch in range(max_epochs):
 
    model.train()
    epoch_loss = 0
    val_loss = 0
    step = 0
    for batch_data in train_loader:
        step += 1
        input_dce, input_adc, labels = batch_data["image_dce"].to(device), batch_data['image_adc'].to(device), batch_data["label"].to(device)
        optimizer.zero_grad()
        outputs = model(input_dce, input_adc)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size

    epoch_loss /= step
    print(f"epoch {epoch + 1} average  train loss: {epoch_loss:.4f}")
    writer.add_scalar('loss/train_loss', epoch_loss, epoch)
    
    if (epoch + 1) % val_interval == 0:
       
        model.eval()
        with torch.no_grad():
            y_pred = torch.tensor([], dtype=torch.float32, device=device)
            y = torch.tensor([], dtype=torch.long, device=device)
            
            step2 = 0
            for val_data in val_loader:
                step2 += 1
                val_dce, val_adc, val_labels = val_data["image_dce"].to(device),val_data["image_adc"].to(device), val_data["label"].to(device)
                val_output = model(val_dce, val_adc)
                y_pred = torch.cat([y_pred, val_output], dim=0)
                y = torch.cat([y, val_labels], dim=0)
                val_loss += loss_function(val_output, val_labels).item()
                
            val_loss /= step2
            writer.add_scalar('loss/val_loss', val_loss, epoch)
            acc_value = torch.eq(y_pred.argmax(dim=1), y)
            acc_metric = acc_value.sum().item() / len(acc_value)
            y_onehot = [post_label(i) for i in decollate_batch(y, detach=False)]
            y_pred_act = [post_pred(i) for i in decollate_batch(y_pred)]
            auc_metric(y_pred_act, y_onehot)
            auc_result = auc_metric.aggregate()
            auc_metric.reset()
            del y_pred_act, y_onehot
            if auc_result > best_metric:
                best_metric = auc_result
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), "best_metric_model_classification3d_dict.pth")
                print("saved new best metric model")
            print(
                "current epoch: {} current accuracy: {:.4f} current AUC: {:.4f} best accuracy: {:.4f} at epoch {}".format(
                    epoch + 1, acc_metric, auc_result, best_metric, best_metric_epoch
                )
            )
            writer.add_scalar('metric/val_Acc', acc_metric, epoch)
            writer.add_scalar('metric/val_AUC', auc_result, epoch)
            
print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

epoch 1 average  train loss: 0.1353
saved new best metric model
current epoch: 1 current accuracy: 0.6606 current AUC: 0.7501 best accuracy: 0.7501 at epoch 1
epoch 2 average  train loss: 0.1599
saved new best metric model
current epoch: 2 current accuracy: 0.6697 current AUC: 0.7562 best accuracy: 0.7562 at epoch 2
epoch 3 average  train loss: 0.1439
saved new best metric model
current epoch: 3 current accuracy: 0.6651 current AUC: 0.7877 best accuracy: 0.7877 at epoch 3
epoch 4 average  train loss: 0.1335
saved new best metric model
current epoch: 4 current accuracy: 0.6743 current AUC: 0.7963 best accuracy: 0.7963 at epoch 4
epoch 5 average  train loss: 0.1383
current epoch: 5 current accuracy: 0.6972 current AUC: 0.7811 best accuracy: 0.7963 at epoch 4
epoch 6 average  train loss: 0.1181
current epoch: 6 current accuracy: 0.6927 current AUC: 0.7692 best accuracy: 0.7963 at epoch 4
epoch 7 average  train loss: 0.1472
current epoch: 7 current accuracy: 0.7110 current AUC: 0.7737 best